In [8]:
from gensim.models.word2vec import Word2Vec
from gensim.models.fasttext import FastText
import gensim
import os,json
import multiprocessing
from collections import defaultdict
from tensorflow.python.client import device_lib
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_devices())
cores = multiprocessing.cpu_count()
print("['CPU cores:",cores,"']")
script_path = os.path.dirname(os.path.abspath('__file__'))
stopwords = set(
    ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves',
     'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their',
     'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was',
     'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the',
     'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against',
     'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in',
     'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why',
     'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only',
     'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now'])
STOP=stopwords

['/device:CPU:0', '/device:XLA_GPU:0', '/device:XLA_CPU:0', '/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3']
['CPU cores: 40 ']


In [9]:
def replacewith(input_str, pattern, replaceWith): 
    return input_str.replace(pattern, replaceWith)
def extract_data(data_file,polarity_map,max_len=80):
    data = open(data_file,'r').readlines()
    corpus=[]
    for i in range(0,len(data),3):
        temp={}
        if len(data[i].split("\n")[0])>=max_len:
            temp["sentence"] = data[i].split("\n")[0]
            temp["aspect_term"] = data[i+1].split("\n")[0]
            temp["polarity"] = polarity_map[data[i+2].split("\n")[0]]
            corpus.append(temp)
    return corpus 

In [10]:
polarity_map={"1":"positive","-1":"negative","0":"neutral"}
train_data=script_path+"/data/restaurant/rest_2014_train.txt"
test_data=script_path+"/data/restaurant/rest_2014_test.txt"
custom_emb_path=script_path+"/custom_embeddings/"
train= extract_data(train_data,polarity_map,max_len=0)
test = extract_data(test_data,polarity_map,max_len=0)
print("Number of training sentences: ",len(train))
print("Number of testing sentences: ",len(test))

Number of training sentences:  3699
Number of testing sentences:  1134


In [11]:
def load_sentences(data):
    sentences=[]
    for x in data:
        sent = replacewith(x["sentence"],"$T$",x["aspect_term"])
        sentences.append(sent.split())
    return sentences    

In [12]:
train_sents = load_sentences(train)
test_sents = load_sentences(test)
print("Number of training sentences: ",len(train_sents))
print("Number of testing sentences: ",len(test_sents))

Number of training sentences:  3699
Number of testing sentences:  1134


In [13]:
# train model
'''cbow_model = Word2Vec(train_sents,size=300,window=3,min_count=1,workers=cores)
skip_model = Word2Vec(train_sents,size=300,window=3,min_count=1,workers=cores,sg=1)
ft_model = FastText(train_sents,size=300,window=3,min_count=1,workers=cores,sg=1)
ft_model.save(custom_emb_path+"ft_model.bin")
cbow_model.save(custom_emb_path+'cbow_model.bin')
skip_model.save(custom_emb_path+'skip_model.bin')'''

In [17]:
w2v_model = Word2Vec.load(custom_emb_path+'cbow_model.bin')
ft_model = FastText.load(custom_emb_path+"ft_model.bin")

In [18]:
model=ft_model
print(model)
words = list(model.wv.vocab)
print("\nWords in vocabulary: ")
print(words)

FastText(vocab=4435, size=300, alpha=0.025)

Words in vocabulary: 
['overdone', 'again', 'Grilled', 'moderately', 'Kosher', 'grace', 'tired', 'Alain', 'cheap', 'Abby', "'em", 'megadeth', 'himself', 'Bombay', 'tiled', 'superior', 'revisiting', 'citysearch', '2-person', 'good', 'waters', 'Mare', 'ONE', 'great', '200', 'grab', 'excellence', 'raucus', 'heritage', 'soon', 'well', 'aisle', 'elegant', 'Grocery', 'Japanese', 'alcohol', 'mussaman', 'apologetic', 'feels', 'Chef', 'such', 'words', 'inside-out', 'asbolute', 'marinated', 'where', 'mixed', '!!!', 'jay-z', 'featuring', 'texture', 'decoration', 'lurking', 'appears', 'pretension', 'Gods', 'grow', 'aside', 'pinnacles', 'mains', 'basmati', 'sea', 'Sea', 'These', 'hinting', 'catching', 'swirl', 'employees', 'hold', 'worried', 'Chowder', 'solid', 'per', 'town', 'priced', 'Kennsington', 'marine', 'Godmother', 'creme', 'super', 'implies', 'earth', 'anyones', 'varietys', 'problem', '30', 'staples', 'disappointment', 'experimenting', 'Birthday

In [19]:
##Testing OOV wod vectors and similarity checks
w1='dirty'
print(model.wv.most_similar(positive=w1))
# access vector for OOv word
w2="sasasvdgfsg"
print(model['sentence'])

[('Monday', 0.9963930249214172), ('wrong', 0.996337354183197), ('Plate', 0.9962970018386841), ('Late', 0.9962823390960693), ('cours', 0.9962821006774902), ('heart', 0.9962745904922485), ('pour-your-own', 0.9962595701217651), ('Saturday', 0.9962563514709473), ('heavy', 0.9962325096130371), ('Same', 0.996202290058136)]
[-0.01773092 -0.3080113  -0.5289587   0.29436553 -0.02114287  0.03985438
 -0.0993406   0.20264676 -0.03148663  0.18313012  0.19075564  0.16572249
 -0.07415963  0.07145736 -0.16134894  0.17128366 -0.3770321  -0.5219213
  0.19671847  0.0546744  -0.12960625  0.143391    0.1536946  -0.16782169
  0.3059072  -0.26570055 -0.15548475  0.61265695  0.17597759  0.04132773
  0.10734734 -0.10166425 -0.17075798 -0.24532662 -0.01053284 -0.18816967
 -0.00221714  0.32405722 -0.3150601  -0.20493652  0.13765484  0.45668942
 -0.02560024  0.01912116 -0.13708204 -0.15529403 -0.26201174 -0.06375071
 -0.08799293 -0.24105263 -0.11426369 -0.03450843  0.16974092 -0.24696541
  0.19054057 -0.0383174  

/home/priyansh.agrawal/py3_env/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
-c:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
